In [3]:
import re
import json
import pprint
import os

mode_transportation_order_detail = {"date": None,
                            "carrier_name": None,
                            "mt_no": None,
                            "shipper_company_name": None,
                            "reciver_company_name": None,
                            "shipper_page_number": None,
                            "reciver_page_number": None
}

def data_clean(text):
    try:
        strip_po = [i.strip() for i in text.split('\\n')]
        remove_space = [i for i in strip_po if i not in [' ', '']]
        remove_space_between = [i.split('  ') for i in remove_space]
        cleaned_data = list()
        count = 0
        for i in remove_space_between:
            cleaned_data.append([])
            for j in i:
                if j not in ['']:
                    cleaned_data[count].append(j.strip())
            count += 1
            
        return cleaned_data
    
    
    except Exception as e:
        print("kvt data clean exception")
        print(e)
        return None
    
    
    
f = open("./MODE_Transportation_21.json", "r")
tmp = f.read()
    
tmp2 = data_clean(tmp)
    
def get_lines_between(origtmp, str1, str2):
    start_index = 0
    stop_index = 0
    for ele_nos, ele in enumerate(origtmp):
        for stchar in ele:
            if str1 in stchar.lower():
                start_index = ele_nos

            if str2 in stchar.lower():
                stop_index = ele_nos
                return origtmp[start_index:stop_index+1]
    return([])

def get_date(origtmp):
    '''
    Derives the date
    '''
    #Regex to find 3 digits followed by space, another 3 digits followed by space and 4 digits.
    regex_date1 = r'[\d]{1,2}\s?\/\s?[\d]{1,2}\s?\/\s?[\d]{4}|[A-Z]{1}[a-z]{3,7}\s[\d]{1,2}[,]{1}\s[\d]{4}'
    compiled_re1 = re.compile(regex_date1)
    
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "tender", "sent by")

    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            date1 = compiled_re1.findall(wordstr)
            if len(date1) > 0:
                return(date1[0]) 
            
    return('None')


def get_mt_no(origtmp):
    '''
    Derives the MT No 
    '''
    #Regex to find letters MT no
    regex_mt_no = r'[A-Z]{2}\s\w+\s\#:\s\d{10}'
    compiled_re = re.compile(regex_mt_no)
    
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2,"carrier", "mt reference")
    
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            mt_no = compiled_re.findall(wordstr)
            if len(mt_no) > 0:
                return(mt_no[0])

    return('None')


def get_carrier_name(origtmp):
    '''
    Derives the Carrier Company
    '''
    #Regex to find carrier name
    regex_carrier = r'[A-Z]{1}[a-z]+\s[A-Z]{1}[a-z]+[\s][A-Z]{1}[a-z]+\s[A-Z]{1}[a-z]+\s[A-Z]{3}'
    compiled_re = re.compile(regex_carrier)
    
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "cover page", "grady dixon")
    
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            carrier = compiled_re.findall(wordstr)
            if len(carrier) > 0:
                return(carrier[0]) 
            
    return('')



def get_shipper_company_name(origtmp):
    '''
    Derives senders company name 
    '''
    #Regex to find the comapny that sends the shipment
    regex_shipper_company = r'[A-Z]+\s[A-Z]+'
    compiled_re2 = re.compile(regex_shipper_company) 
    
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "covering mode", "cardboard and plastic")
    
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            name = compiled_re2.findall(wordstr)
            if len(name) > 0:
                return(name[0])
    return('None')



def get_reciver_company_name(origtmp):
    '''
    Derives reciver company name 
    '''
    #Regex to find the comapny that recives the shipment
    
    regex_reciver_company = r'[A-Z]+\s[A-Z]+\s[&]?\s[A-Z]+'
    compiled_re = re.compile(regex_reciver_company) 
    
    #Get the relevant lines for search
    get_lines = get_lines_between(tmp2, "covering mode", "cardboard and plastic")
    
    #Loop through the subset of lines and apply regex
    for line in get_lines:
        for wordstr in line:
            name = compiled_re.findall(wordstr)
            if len(name) > 0:
                return(name[0])
    return('None')



def get_shipper_page_number(origtmp):
    '''
    Derives shipper_page_number 
    '''
    text_1 = 'Acceptance of this load'
    regex_shipper_page_number = r'[A-Za-z]{10}\s[a-z]{2}\s[a-z]{4}\s[a-z]{4}\s[a-z]{2}'
    compiled_re1 = re.compile(regex_shipper_page_number)
    
    get_lines = get_lines_between(tmp2, "customs broker", "stop 1 (shipper)")
    
    for line in get_lines:
        for text_1 in line:
            lc = compiled_re1.findall(text_1)
            if len(lc) > 0:
                return('page 2')
    
    return('None')



def get_reciver_page_number(origtmp):
    '''
    Derives reciver_page_number 
    '''
    text_1 = 'FOOD SAFETY'
    regex_reciver_page_number = r'[A-Z]{4}\s[A-Z]{6}'
    compiled_re3 = re.compile(regex_reciver_page_number)
    
    get_lines = get_lines_between(tmp2, "customs broker", "stop 1 (shipper)")
    
    for line in get_lines:
        for text_1 in line:
            lc = compiled_re3.findall(text_1)
            if len(lc) > 0:
                return('page 3')
    
    return('None')



def get_mode_transportation_info(tmp2):
        
    date = get_date(tmp2)
    carrier_name = get_carrier_name(tmp2)
    mt_no = get_mt_no(tmp2)
    shipper_company_name = get_shipper_company_name(tmp2)
    reciver_company_name = get_reciver_company_name(tmp2)
    shipper_page_number = get_shipper_page_number(tmp2)
    reciver_page_number = get_reciver_page_number(tmp2)
    
    
    mode_transportation_order_detail["date"] = date
    mode_transportation_order_detail["carrier_name"] = carrier_name
    mode_transportation_order_detail["mt_no"] = mt_no
    mode_transportation_order_detail["shipper_company_name"] = shipper_company_name
    mode_transportation_order_detail["reciver_company_name"] = reciver_company_name
    mode_transportation_order_detail["shipper_page_number"] = shipper_page_number 
    mode_transportation_order_detail["reciver_page_number"] = reciver_page_number
    
    #get_mode_transportation_info(tmp2)
    
    #pprint.pprint(mode_transportation_order_detail)
    
    

In [2]:
for i in range(21, 40):
    mode_transportation_order_detail = {"date": None,
                                    "carrier_name": None,
                                    "mt_no": None,
                                    "shipper_company_name": None,
                                    "reciver_company_name": None,
                                    "shipper_page_number": None,
                                    "reciver_page_number": None
    }
    filename = f'./MODE_Transportation_{i}.json'
    with open(filename) as f:
        tmp2 = data_clean(f.read())
        get_mode_transportation_info(tmp2)
        pprint.pprint(mode_transportation_order_detail)
        
        

{'carrier_name': 'Fidelis Freight Transport Services LLC',
 'date': '04/23/2021',
 'mt_no': 'MT Reference #: 2930129302',
 'reciver_company_name': 'L FISHMAN & SON',
 'reciver_page_number': 'page 3',
 'shipper_company_name': 'RAM BOARD',
 'shipper_page_number': 'page 2'}
{'carrier_name': 'Fidelis Freight Transport Services LLC',
 'date': '05/12/2021',
 'mt_no': 'MT Reference #: 2930131077',
 'reciver_company_name': 'None',
 'reciver_page_number': 'page 3',
 'shipper_company_name': 'None',
 'shipper_page_number': 'page 2'}
{'carrier_name': 'Fidelis Freight Transport Services LLC',
 'date': '07/09/2020',
 'mt_no': 'None',
 'reciver_company_name': 'None',
 'reciver_page_number': 'None',
 'shipper_company_name': 'None',
 'shipper_page_number': 'page 2'}
{'carrier_name': 'Fidelis Freight Transport Services LLC',
 'date': '05/17/2021',
 'mt_no': 'None',
 'reciver_company_name': 'None',
 'reciver_page_number': 'page 3',
 'shipper_company_name': 'None',
 'shipper_page_number': 'page 2'}
{'carr